In [13]:

# Imports
import torch
import numpy as np
import pandas as pd
import pickle
import os
import time
import torch.nn as nn
import subprocess
import time
import threading

from typing import Dict, List

from torch.utils.data import DataLoader
from helper.config import TrainConfig, SuspiciousnessConfig, ExperimentConfig, ExperimentRun

from helper.global_variables import TRAIN_YAML_PATH, SUS_YAML_PATH, EXPERIMENT_YAML_PATH
from helper.global_variables import WORKERS as NUM_WORKERS
from helper.general import ensure_directory_exists
from helper.gradients import perform_gradient_descent
from helper.datasets import get_dataset_loader
from helper.models import get_model, get_sub_model, evaluate_model_per_class
from helper.general import empty_gpu_cache
from helper.plot import plot_images, show_bar_diagram

torch.set_warn_always(False)

In [14]:
# Load General Config
train_config: TrainConfig = TrainConfig(TRAIN_YAML_PATH)
sus_config: SuspiciousnessConfig = SuspiciousnessConfig(SUS_YAML_PATH)
experiment_config: ExperimentConfig = ExperimentConfig(EXPERIMENT_YAML_PATH, train_config)

In [15]:
# Results Helper Functions

def get_sus_path(model_name, layer_config):
    return f"model-sus-values/{model_name}/layer-config-{layer_config}.pickle"

class Result:
    def __init__(self,
                 predictions_count: Dict[str, float], 
                 duration: float,
                 target_class: str,
                 ratio_target_classifications: float,
                 model_accuracy: float, 
                 exp_run: ExperimentRun):
        
        self.experiment_name = exp_run.experiment_name
        self.scenario_name = exp_run.scenario_name
        self.target_class = target_class
        self.duration = duration
        self.predictions_count = predictions_count
        self.ratio_target_classifications = ratio_target_classifications
        self.model_accuracy = model_accuracy
        self.exp_run: ExperimentRun = exp_run
        
    def __str__(self):
        log = f"Experiment: {self.experiment_name}\n"
        log += f", Scenario: {self.scenario_name}\n"
        log += f", Target Class: {self.target_class}\n"
        log += f", Ratio Target Classifications: {self.ratio_target_classifications}\n"
        log += f", Duration: {self.duration}\n"
        log += f", Model Accuracy: {self.model_accuracy}\n"
        return log

results: List[Result] = []

def save_results(results: List[Result], output_folder: str):
    # Convert results to dict format
    result_dicts = []
    for result in results:
        result_dict = {
            'model_name': result.exp_run.model_name,
            'experiment': f"{result.experiment_name.replace("experiment-", "")}",
            'scenario': f"{result.scenario_name.replace("case-", "")}",
            'sus_metric': result.exp_run.sus_metric,
            'samples': result.exp_run.samples,
            'num_neurons': result.exp_run.num_neurons,
            'grad_factor': result.exp_run.grad_factor,
            'num_iterations': result.exp_run.iterations,
            'layer_config': result.exp_run.layer_config,
            'target_predictions': f"{result.ratio_target_classifications:.2f}% (Class {result.target_class})",
            'accuracy': f"{result.model_accuracy:.2f} %",
            'duration': f"{result.duration:.2f} s",
        }
        result_dicts.append(result_dict)
    
    df = pd.DataFrame(result_dicts)
    if df.empty:
        raise ValueError("No valid results to save")
        
    df = df.sort_values(['model_name', 'experiment', 'scenario'])
    
    df.to_csv(os.path.join(output_folder, 'results.csv'), index=False)
    return df


In [16]:
# GPU Monitoring Helper
def ensure_file_exists(file_path: str):
    if not os.path.exists(file_path):
        with open(file_path, "w+") as f:
            f.write("")

def run_nvidia_smi():
    result = subprocess.run(['nvidia-smi', 
                             "--query-gpu=utilization.gpu,memory.used,memory.total", 
                             "--format=csv,nounits,noheader",
                             "--id=0"], 
                            stdout=subprocess.PIPE)
    return result.stdout.decode('utf-8')

def gpu_monitoring():
    global stop_monitoring
    global gpu_monitoring_file
    global start_time
    with open(gpu_monitoring_file, "a+") as f:
        f.write(f"time,gpu_utilization,memory_used,memory.total\n")
        while not stop_monitoring:
            line=run_nvidia_smi()
            f.write(f"{int(start_time) - int(time.time())},{line}")
            time.sleep(0.25)

In [17]:
# Helper Functions
def load_sus_values(exp_run: ExperimentRun):
    sus_path = get_sus_path(exp_run.model_name, exp_run.layer_config)
    with open(sus_path, "rb") as f:
        if exp_run.sus_metric == "ochiai":
            (sus_values, _) = pickle.load(f)
        elif exp_run.sus_metric == "tarantula":
            (_, sus_values) = pickle.load(f)
        else:
            raise Exception("Invalid Suspiciousness Metric")

    print(f"Suspiciousness ({exp_run.sus_metric}) value range: {
          sus_values.min():.4f} to {sus_values.max():.4f}")
    print(f"NaN in Susp. values ({exp_run.sus_metric}): {
          np.isnan(sus_values).any()}")
    return sus_values


def generate_adv_input(model: nn.Module,
                       dataset_loader: DataLoader,
                       exp_run: ExperimentRun,
                       sus_values: np.ndarray) -> DataLoader:
    sub_model = get_sub_model(model, exp_run.model_name, exp_run.layer_config)

    modified_images = perform_gradient_descent(model=sub_model,
                                               dataset_loader=dataset_loader,
                                               sus_values=sus_values,
                                               num_neurons=exp_run.num_neurons,
                                               num_iterations=exp_run.iterations,
                                               grad_factor=exp_run.grad_factor)

    modified_loader = DataLoader(modified_images,
                                 batch_size=sus_config.batch_size,
                                 shuffle=False,
                                 num_workers=NUM_WORKERS)
    return modified_loader


def evaluate_experimental_result(model: nn.Module,
                                 loader: DataLoader,
                                 exp_run: ExperimentRun,
                                 duration: float,
                                 show_bar_diagram: bool=False) -> Result:

    pred_dict_mod, acc_mod = evaluate_model_per_class(model, loader)
    print(f"Pred Dict Mod: {pred_dict_mod}")
    
    if show_bar_diagram:    
        values = pred_dict_mod.values()
        labels = pred_dict_mod.keys()
        show_bar_diagram(labels, values)

    return Result(duration=duration,
                  predictions_count=pred_dict_mod,
                  target_class=sus_config.target_class,
                  ratio_target_classifications=pred_dict_mod[sus_config.target_class] / len(
                      loader),
                  model_accuracy=acc_mod,
                  exp_run=exp_run)


def save_images(exp_run: ExperimentRun,
                modified_images: torch.Tensor):
    target_path = f"{exp_run.result_dir}{exp_run.experiment_name}"
    ensure_directory_exists(target_path)
    plot_images(modified_images, 4, target_path +
                f"/{exp_run.scenario_name}-{exp_run.model_name}.png")

In [ ]:
# TODO should be moved somewhere else

# Keep track of covered datasets to store original images only once
covered_datasets = []

# Experiment Run
for idx, exp_run in enumerate(experiment_config.exp_runs):

    print("--- " * 10 + f"Experiment Run {idx + 1} of {len(experiment_config.exp_runs)}" + " --- " * 10)
    print(exp_run)
    
    # Set environment variable for gpu usage script
    stop_monitoring = False
    gpu_monitoring_file = f"{exp_run.result_dir}{exp_run.experiment_name}-{exp_run.scenario_name}-gpu-usage.csv"
    ensure_directory_exists(exp_run.result_dir)
    ensure_file_exists(gpu_monitoring_file)
    print(f"GPU Monitoring File: {gpu_monitoring_file}")
    monitor_thread = threading.Thread(target=gpu_monitoring)
    monitor_thread.start()
    try:
        # TODO use wrapper function
        model = get_model(exp_run.model_name)
        model_dict = torch.load(exp_run.model_path, weights_only=True)
        model.load_state_dict(model_dict)

        # TODO use helper function to load relevant portion of dataset
        # Load Dataset
        _, mod_dataset_loader = get_dataset_loader(exp_run.dataset_name)
        dataset_orig = mod_dataset_loader.dataset
        indices = [idx for idx in range(len(dataset_orig)) if dataset_orig[idx][1] != sus_config.target_class]

        # Check length of filtered dataset to avoid samples number greater than original dataset
        if len(indices) < exp_run.samples:
            raise Exception(f"Filtered dataset is smaller than given sample size: {len(indices)} vs {exp_run.samples}")

        # Create filtered subset containing only samples not of target class
        filtered_dataset = torch.utils.data.Subset(dataset_orig, indices)
        mod_dataset_loader = DataLoader(dataset_orig, batch_size=sus_config.batch_size, shuffle=False, num_workers=NUM_WORKERS)

        sus_values = load_sus_values(exp_run)

        # Main Adversarial Input Generation (Time Tracking)
        time_pre = time.perf_counter()
        modified_images_loader = generate_adv_input(model, mod_dataset_loader, exp_run, sus_values)
        time_diff = time.perf_counter() - time_pre

        # Evaluate Experimental Result
        results.append(evaluate_experimental_result(model,
                                                    modified_images_loader,
                                                    exp_run,
                                                    time_diff))

        print(f"Duration: {time_diff:.2f} s")

        # TODO: Plot images, flag must become part of config.py
        if exp_run.print_images:

            save_images(exp_run, modified_images_loader.dataset)

        empty_gpu_cache()
    except Exception as e:
        print(f"Error in Experiment Run {idx + 1} of {len(experiment_config.exp_runs)}")
        print(e)
        results.append(Result(predictions_count=None, 
                              duration=None, 
                              target_class=None, 
                              ratio_target_classifications=None, 
                              model_accuracy=None, 
                              exp_run=exp_run))
        empty_gpu_cache()
    stop_monitoring = True
    monitor_thread.join()
    
output_folder = f"{experiment_config.exp_runs[0].result_dir}"
ensure_directory_exists(output_folder)
save_results(results, output_folder)

--- --- --- --- --- --- --- --- --- --- Experiment Run 1 of 58 ---  ---  ---  ---  ---  ---  ---  ---  ---  --- 
Experiment(experiment-layer-variations)/Scenario(case-0) on model cifar10-mobilenet
- Layer Config: 1
- Susp. Metric: ochiai
- Number of Neurons: 5
- Number of Iterations: 5
- Number of Sample: 9000
- Gradient Factor: 5

Directory created: results-adv-gen/
GPU Monitoring File: results-adv-gen/experiment-layer-variations-case-0-gpu-usage.csv


/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Suspiciousness (ochiai) value range: 0.0313 to 0.1842
NaN in Susp. values (ochiai): 0


Evaluate Model ...: 100%|██████████| 157/157 [00:06<00:00, 23.87it/s]


Pred Dict Mod: {3: 1318, 8: 677, 0: 1378, 6: 1014, 1: 1180, 9: 850, 5: 927, 7: 948, 4: 890, 2: 818}
Duration: 12.64 s
Directory created: results-adv-gen/experiment-layer-variations
--- --- --- --- --- --- --- --- --- --- Experiment Run 2 of 58 ---  ---  ---  ---  ---  ---  ---  ---  ---  --- 
Experiment(experiment-layer-variations)/Scenario(case-0) on model cifar10-squeezenet
- Layer Config: 1
- Susp. Metric: ochiai
- Number of Neurons: 5
- Number of Iterations: 5
- Number of Sample: 9000
- Gradient Factor: 5

Directory already exists: results-adv-gen/
GPU Monitoring File: results-adv-gen/experiment-layer-variations-case-0-gpu-usage.csv


/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/d.rashedi/miniconda3/envs/py12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified
